<a href="https://colab.research.google.com/github/lani009/IDS-DataMining/blob/main/%5BDM%5DApriori.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time

In [2]:
data = pd.read_csv('DM_data.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25192 entries, 0 to 25191
Data columns (total 40 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   duration                     25192 non-null  int64  
 1   protocol_type                25192 non-null  int64  
 2   service                      25192 non-null  int64  
 3   flag                         25192 non-null  int64  
 4   src_bytes                    25192 non-null  int64  
 5   dst_bytes                    25192 non-null  int64  
 6   land                         25192 non-null  int64  
 7   wrong_fragment               25192 non-null  int64  
 8   hot                          25192 non-null  int64  
 9   num_failed_logins            25192 non-null  int64  
 10  logged_in                    25192 non-null  int64  
 11  num_compromised              25192 non-null  int64  
 12  root_shell                   25192 non-null  int64  
 13  su_attempted    

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [4]:
from mlxtend.frequent_patterns import apriori,association_rules

In [5]:
sc = StandardScaler() 
sc_data = sc.fit_transform(data)

sc_df = pd.DataFrame(sc_data, columns=data.columns)
sc_df.head(n=10)

#StandardScaler로 data scaling

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,hot,num_failed_logins,logged_in,num_compromised,root_shell,su_attempted,num_root,num_file_creations,num_shells,num_access_files,is_guest_login,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,class,index_num
0,-0.113551,-0.444009,-1.399448,0.744553,-0.009889,-0.039310,-0.00891,-0.091223,-0.091933,-0.02622,-0.807626,-0.021873,-0.039377,-0.027665,-0.021724,-0.027808,-0.018905,-0.043917,-0.09599,-0.720244,-0.354628,-0.640142,-0.633978,-0.372186,-0.373098,0.772109,-0.349282,-0.373886,-0.328634,-0.813985,-0.779157,-0.280673,0.073120,-0.287993,-0.641804,-0.627365,-0.221668,-0.374281,-0.934425,-1.731982
1,-0.113551,1.325565,0.780883,0.744553,-0.010032,-0.039310,-0.00891,-0.091223,-0.091933,-0.02622,-0.807626,-0.021873,-0.039377,-0.027665,-0.021724,-0.027808,-0.018905,-0.043917,-0.09599,-0.624317,-0.368427,-0.640142,-0.633978,-0.372186,-0.373098,-1.320567,0.490836,-0.373886,0.732059,-1.030895,-1.157831,2.764403,2.375620,-0.287993,-0.641804,-0.627365,-0.385140,-0.374281,-0.934425,-1.731845
2,-0.113551,-0.444009,-1.377199,-0.917300,-0.010093,-0.039310,-0.00891,-0.091223,-0.091933,-0.02622,-0.807626,-0.021873,-0.039377,-0.027665,-0.021724,-0.027808,-0.018905,-0.043917,-0.09599,0.334947,-0.299430,1.595477,1.600209,-0.372186,-0.373098,-1.388806,0.042773,-0.373886,0.732059,-0.804947,-0.935081,-0.173828,-0.478183,-0.287993,1.603834,1.614454,-0.385140,-0.374281,1.070177,-1.731707
3,-0.113551,-0.444009,0.780883,0.744553,-0.009996,0.052473,-0.00891,-0.091223,-0.091933,-0.02622,1.238197,-0.021873,-0.039377,-0.027665,-0.021724,-0.027808,-0.018905,-0.043917,-0.09599,-0.694082,-0.313230,-0.193018,-0.187141,-0.372186,-0.373098,0.772109,-0.349282,-0.373886,-1.540854,1.264742,1.069663,-0.440940,-0.380894,0.073759,-0.574435,-0.604947,-0.385140,-0.342768,-0.934425,-1.731570
4,-0.113551,-0.444009,0.780883,0.744553,-0.010010,-0.034582,-0.00891,-0.091223,-0.091933,-0.02622,1.238197,-0.021873,-0.039377,-0.027665,-0.021724,-0.027808,-0.018905,-0.043917,-0.09599,-0.476067,0.059355,-0.640142,-0.633978,-0.372186,-0.373098,0.772109,-0.349282,-0.023115,0.732059,1.264742,1.069663,-0.440940,-0.478183,-0.287993,-0.641804,-0.627365,-0.385140,-0.374281,-0.934425,-1.731432
5,-0.113551,-0.444009,-1.377199,-2.025203,-0.010093,-0.039310,-0.00891,-0.091223,-0.091933,-0.02622,-0.807626,-0.021873,-0.039377,-0.027665,-0.021724,-0.027808,-0.018905,-0.043917,-0.09599,0.317506,-0.120038,-0.640142,-0.633978,2.765176,2.729322,-1.138595,-0.013235,-0.373886,0.732059,-0.868212,-1.001906,-0.066984,-0.478183,-0.287993,-0.641804,-0.627365,2.884296,2.777041,1.070177,-1.731295
6,-0.113551,-0.444009,-1.377199,-0.917300,-0.010093,-0.039310,-0.00891,-0.091223,-0.091933,-0.02622,-0.807626,-0.021873,-0.039377,-0.027665,-0.021724,-0.027808,-0.018905,-0.043917,-0.09599,0.709933,-0.258032,1.595477,1.600209,-0.372186,-0.373098,-1.388806,-0.013235,-0.373886,0.732059,-0.958592,-1.068731,-0.173828,-0.478183,-0.287993,1.603834,1.614454,-0.385140,-0.374281,1.070177,-1.731157
7,-0.113551,-0.444009,-1.377199,-0.917300,-0.010093,-0.039310,-0.00891,-0.091223,-0.091933,-0.02622,-0.807626,-0.021873,-0.039377,-0.027665,-0.021724,-0.027808,-0.018905,-0.043917,-0.09599,0.282624,-0.161436,1.595477,1.600209,-0.372186,-0.373098,-1.184088,-0.013235,-0.373886,0.732059,-0.904364,-1.024181,-0.066984,-0.478183,-0.287993,1.603834,1.614454,-0.385140,-0.374281,1.070177,-1.731019
8,-0.113551,-0.444009,0.780883,-0.917300,-0.010093,-0.039310,-0.00891,-0.091223,-0.091933,-0.02622,-0.807626,-0.021873,-0.039377,-0.027665,-0.021724,-0.027808,-0.018905,-0.043917,-0.09599,1.616874,-0.064840,1.595477,1.600209,-0.372186,-0.373098,-1.297820,-0.069243,-0.373886,0.732

In [6]:
def encode_units(x):
  if x <= 0 :
    return 0
  if x > 0 :
    return 1

train_df = sc_df.applymap(encode_units)

train_df.head(n=10)

#classification을 위해 scaling 시킨 data들을 음수면 0, 양수면 1로 encoding

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,hot,num_failed_logins,logged_in,num_compromised,root_shell,su_attempted,num_root,num_file_creations,num_shells,num_access_files,is_guest_login,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,class,index_num
0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,1,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,0,1,0,1,0,0,0,0,0,1,1,0,0,1,0
3,0,0,1,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,1,0,0,0,0,0,0
4,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,1,1,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,1,1,1,0
6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,1,0
7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,1,0
8,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,1,0
9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,1,0


In [7]:
data_X = train_df.drop(columns = ["index_num"])

X_train, X_test = train_test_split(data_X, test_size=0.3, shuffle=True)
print(X_train.shape, X_test.shape)

#train data와 test data를 7:3 의 비율로 split

(17634, 39) (7558, 39)


In [8]:
df = pd.DataFrame(X_train, columns=data.drop(columns = ["index_num"]).columns)

df.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,hot,num_failed_logins,logged_in,num_compromised,root_shell,su_attempted,num_root,num_file_creations,num_shells,num_access_files,is_guest_login,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,class
21843,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,1,0,1,0,0,0,0,0,1
10713,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,1
1773,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,1,1,0,0,0,0,0,0,0,0
6797,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,1,0,0,0,0,0,0,0,0
23799,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,1,0,0,0,0,0,1,1,0,0,1


In [9]:
frequent_itemsets = apriori( df, min_support = 0.1, use_colnames=True, max_len = 2)

result_desc = frequent_itemsets.sort_values(['support'],ascending =[False])
result_desc

,support,itemsets
13,0.644267,(dst_host_count)
10,0.623001,(same_srv_rate)
1,0.617444,(service)
2,0.609221,(flag)
45,0.568334,"(flag, same_srv_rate)"
...,...,...
79,0.101962,"(class, rerror_rate)"
82,0.101565,"(class, srv_rerror_rate)"
27,0.101225,"(protocol_type, dst_host_srv_count)"
29,0.100488,"(protocol_type, dst_host_same_src_port_rate)"


In [10]:
rules = association_rules(result_desc , metric = "confidence" , min_threshold = 0.8)
rules = rules.sort_values(['confidence','lift'], ascending=[False , False])
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
40,(protocol_type),(flag),0.184757,0.609221,0.184757,1.000000,1.641441,0.072199,inf
50,(srv_count),(flag),0.131507,0.609221,0.130997,0.996119,1.635071,0.050880,100.690768
17,(srv_serror_rate),(serror_rate),0.284110,0.286832,0.282919,0.995808,3.471745,0.201427,170.141448
20,(dst_host_srv_serror_rate),(dst_host_serror_rate),0.279403,0.285585,0.276738,0.990461,3.468186,0.196945,74.892007
51,(srv_count),(same_srv_rate),0.131507,0.623001,0.130203,0.990082,1.589214,0.048274,38.011332
...,...,...,...,...,...,...,...,...,...
37,(dst_host_serror_rate),(count),0.285585,0.364920,0.232108,0.812748,2.227195,0.127893,3.391583
68,(rerror_rate),(class),0.125496,0.466258,0.101962,0.812472,1.742536,0.043448,2.846193
36,(serror_rate),(count),0.286832,0.364920,0.232449,0.810399,2.220759,0.127778,3.349567
69,(srv_rerror_rate),(class),0.126460,0.466258,0.101565,0.803139,1.722519,0.042602,2.711262


In [11]:
rules_list = rules[rules['consequents'] == {"class"}]
rules_list

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
31,(dst_host_srv_serror_rate),(class),0.279403,0.466258,0.274980,0.984169,2.110780,0.144706,33.714680
30,(srv_serror_rate),(class),0.284110,0.466258,0.275264,0.968862,2.077952,0.142795,17.141318
29,(dst_host_serror_rate),(class),0.285585,0.466258,0.275320,0.964059,2.067649,0.142164,14.850403
22,(serror_rate),(class),0.286832,0.466258,0.275831,0.961645,2.062472,0.142093,13.915800
15,(count),(class),0.364920,0.466258,0.323409,0.886247,1.900764,0.153262,4.692114
48,(diff_srv_rate),(class),0.170296,0.466258,0.145911,0.856810,1.837629,0.066509,3.727503
68,(rerror_rate),(class),0.125496,0.466258,0.101962,0.812472,1.742536,0.043448,2.846193
69,(srv_rerror_rate),(class),0.126460,0.466258,0.101565,0.803139,1.722519,0.042602,2.711262


In [12]:
test = pd.DataFrame(X_test, columns=data.drop(columns = ["index_num"]).columns)
test.head(n=10)

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,hot,num_failed_logins,logged_in,num_compromised,root_shell,su_attempted,num_root,num_file_creations,num_shells,num_access_files,is_guest_login,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,class
3445,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,1,0,0,0,0,0,0,0,0
17461,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,1,0,0,0,0,0,0,0,0
14662,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,1
4043,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,1,0,1,0,0,0,0,0,0,0,1,1,1
9161,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0
3396,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,1,0,0,1,1,0,0,0,1,1,1
16768,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,1,0,0,0,0,0,0,0,0
19271,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0
11404,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,1,0,0,0,0,0,0,0,0
10421,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,1,1,0,0,0,0,0,0,0,0


In [ ]:
col = ['dst_host_srv_serror_rate', 'srv_serror_rate', 'serror_rate', 'dst_host_serror_rate', 'count', 'diff_srv_rate', 'rerror_rate', 'srv_rerror_rate']

In [13]:
idx_a = test[(test['dst_host_srv_serror_rate'] == 0) & (test['srv_serror_rate'] == 0) & (test['serror_rate'] == 0) & (test['dst_host_serror_rate'] == 0) & (test['count'] == 0) & (test['diff_srv_rate'] == 0) & (test['rerror_rate'] == 0) & (test['srv_rerror_rate'] == 0) ].index
test_df = test.drop(idx_a) #attack 예측

idx_class = test_df[test_df['class'] == 0 ].index
test_err = test_df.drop(idx_class) #attack 결과

print(test_df.shape)
print(test_err.shape)

(3927, 39)
(3072, 39)


In [14]:
idx_a = test[(test['dst_host_srv_serror_rate'] == 1) | (test['srv_serror_rate'] == 1) | (test['serror_rate'] == 1) | (test['dst_host_serror_rate'] == 1) | (test['count'] == 1) | (test['diff_srv_rate'] == 1) | (test['rerror_rate'] == 1) | (test['srv_rerror_rate'] == 1)].index
test_df = test.drop(idx_a) #non 이라고 예측

idx_class = test_df[test_df['class'] == 1 ].index
test_err = test_df.drop(idx_class) #non 결과

print(test_df.shape)
print(test_err.shape)

(3631, 39)
(3182, 39)


min_support = 0.1 / max_len = 2 / min_threshold = 0.8 

 ('dst_host_srv_serror_rate', 'srv_serror_rate', 'serror_rate', 'dst_host_serror_rate', 'count', 'diff_srv_rate', 'rerror_rate', 'srv_rerror_rate')



               |   Prediction of Attack     |  Prediction of Non-Attack
---
    Attack     |    True Positive : 3072    |    False Negative : 449
---
    Non-Attack |    False Positive :  855   |    True Negative : 3182



**Apriori Test**

*  Accuracy (82.75)
* Precision (78.23)
* Recall (87.25)
* F1 score (82.49)
